### 다음 이미지 크롤러 함수 사용시 주의사항
#### 2020-04-10(금) 이한울
---
- 가끔씩 next_key 변수 위치를 찾을 수 없어서 elementException 에러가 발생하는데 그 떄에는 몇 번 다시 실행하면 정상적으로 작동합니다.
- 해당 문제를 해결하기 위해 지속적으로 방법을 찾아보고 있습니다.
- 네이버와는 다르게 다음은 상세검색을 지원하지 않기 때문에 따로 제외 키워드와 검색할 키워드 리스트를 작성하지는 않았습니다.
---
#### 2020-04-13 (월) 이한울
---
- 네이버에서 일부 이미지가 누락되어 저장되는 현상은 다음에서는 발생되지 않았습니다.
- 따라서 다음 이미지 크롤링 함수는 정상적으로 사용할 수 있다고 판단됩니다.
---
#### 2020-04-16 (목) 이한울
---
- 다음의 검색 키워드 리스트인 d_mk_list와 d_fk_list를 아래 셀에 작성해놓았습니다.
- 해당 리스트를 for문을 태워서 검색어 하나씩 이미지 크롤링을 실행해주시면 됩니다!
---
#### 2020-05-07 (목) 서동원
---
- 한울코드 + 존잘코드 퓨전!
- 최하단 'Main cell' 아래에 있는 셀만 실행하시면 됩니다.
- tqdm 구현은 못함...
- 현재 노트북 파일이 있는 위치에 download 폴더와 그 하위에 검색어별 폴더가 자동 생성됩니다.

In [16]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import time
import tqdm
import os
import shutil

import selenium.common.exceptions as sce
from queue import Queue
from multiprocessing.pool import ThreadPool

In [17]:
def make_dir(dirname):
    current_path = os.getcwd()
    path = os.path.join(current_path, dirname)
    if not os.path.exists(path):
        os.makedirs(path)

In [18]:
def wait_and_click(driver, xpath):
    try:
        wait = WebDriverWait(driver, 10)
        elem = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
        elem.click()
    except Exception:
        print('click error..')
        print('browser refreshing')
        driver.refresh()
        time.sleep(2.5)
        return wait_and_click(driver, xpath)
    
    return 

In [19]:
def get_scroll(driver):
    pos = driver.execute_script("return window.pageYOffset;")
    return pos

In [20]:
def collect_img_url(driver, body, thread_num, keyword):
    srcs = []

    count=0
    last_scroll = 0
    scroll_patience = 0

    while 1:
        try:
            xpath_inner_thum = '//div[@class="inner_thumb"]//img[@class="thumb_img"]'
            imgs = driver.find_elements(By.XPATH, xpath_inner_thum)

            src = imgs[1].get_attribute('src')

            if src not in srcs and src is not None:
                srcs.append(src)
                print(f'[thread {thread_num}: {keyword}]\n{count}: {src}\n')
                count += 1

        except sce.StaleElementReferenceException:
            print('[StaleElementReferenceException]')
            pass
        except IndexError:
            print('[IndexError]')
            break

        scroll = get_scroll(driver)
        if scroll == last_scroll:
            scroll_patience += 1
        else:
            scroll_patience = 0
            last_scroll = scroll

        if scroll_patience >= 20:
            try:
                xpath_more_btn = '//div[@class="extend_comp extend_imgtab"]//a[@class="expender open"]'
                elem = driver.find_element(By.XPATH, xpath_more_btn)
                if elem.is_displayed():
                    elem.click()
                    time.sleep(2.5)
                else:
                    break
            except sce.NoSuchElementException:
                break
            
            
        # 다음 사진으로 넘기기
        body.send_keys(Keys.RIGHT)
    
    return list(set(srcs))  # 중복 제거 후 반환

In [21]:
def get_ext(src):
    ext = src.split('.')[-1].lower()
    ext = ext.split('?', 1)[0]
    
    if ext == 'jpg' or ext == 'jpeg':
        ext = 'jpg'
    elif ext == 'png':
        ext = 'png'
    elif ext == 'gif':
        ext = 'gif'
    else:
        ext = 'jpg'
    
    return ext

In [22]:
def save_img(response, file_path):
    
    with open(file_path, 'wb') as file:
        shutil.copyfileobj(response.raw, file)
    del response

    

In [23]:
def all_files(dir_path):
    paths = []
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            if os.path.isfile(dir_path + '/' + file):
                paths.append(dir_path + '/' + file)
    return paths

In [24]:
def validate_img(dir_path):
    paths = all_files(dir_path)
    
    invalid_paths = list(filter(lambda x: len(x.split('.')[-1]) > 4, paths))
    for path in invalid_paths:
        os.remove(path)
        print(f'Invalid file deleted: {path}')
    
    jpeg_paths = list(filter(lambda x: len(x.split('.')[-1]) == 'jpeg', paths))
    for path in jpeg_paths:
        new_path = path.split('.')[0] + 'jpg'
        os.rename(path, new_path)
        print('File name renamed:')
        print(f"'{path}'\nto\n'{new_path}'\n")

In [25]:
def do_daum_search(driver_path, keyword, thread_num):
    
    os.makedirs('./download', exist_ok=True)
    
    driver = Chrome(driver_path)
    session = requests.Session()

    # 요청 보낼 때 header와 함께 전송
    header = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
               AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36", 
              "Accept":"text/html,application/xhtml+xml,application/xml;\
              q=0.9,imgwebp,*/*;q=0.8"}
    
    # 검색 url
    #naver_image =  'https://search.naver.com/search.naver?where=image&sm=tab_jum&query=' + keyword
    daum_img_search = 'https://search.daum.net/search?w=img&nil_search=btn&DA=NTB&enc=utf8&q=' + keyword
    
    # 세션 get
    req = session.get(daum_img_search, headers=header)
    
    # 드라이버 get
    driver.get(daum_img_search)
    time.sleep(1)
    
    # body elem 변수 저장
    body = driver.find_element(By.TAG_NAME, 'body')
    
    # 첫번째 사진 클릭
    xpath_outer_thum = '//div[@class="wrap_thumb"]'
    wait_and_click(driver, xpath_outer_thum)
    
    # 이미지 링크 수집
    print('*' * 30
          + f'\n[thread {thread_num}: {keyword}]\nCollecting image start !!\n'
          + '*' * 30, end='\n')
    links = collect_img_url(driver, body, thread_num, keyword)
    print('*' * 30
          + f'\n[thread {thread_num}: {keyword}]\nCollecting image links success : {len(links)} links\n'
          + '*' * 30, end='\n')
    
    # 세션 및 드라이버 종료
    session.close()
    driver.quit()
    
    # 디렉토리 생성
    dir_path = f'./download/{keyword}'
    make_dir(dir_path)
    
    # 이미지 저장
    print('*' * 30
          + f'\n[thread {thread_num}: {keyword}]\nSaving image start !!\n'
          + '*' * 30, end='\n')
    for i, link in enumerate(links):
        
        try:
            response = requests.get(link, stream=True)
            print(f'[thread {thread_num}: {keyword}]\n{i}: image downloaded\n')
        except Exception as e:
            print(f'[thread {thread_num}: {keyword}]\n{i}: download failed\n', e)
            continue
        try:    
            ext = get_ext(link)
            file_without_ext = f'daum_{keyword}_{str(i).zfill(4)}'
            file_path = f'{dir_path}/{file_without_ext}.{ext}'
            save_img(response, file_path) 
            print(f'[thread {thread_num}: {keyword}]\n{i}: image saved\n')
        except Exception as e:
            print(f'[thread {thread_num}: {keyword}]\n{i}: save failed\n', e)
    print('*' * 30
          + f'\n[thread {thread_num}: {keyword}]\nSaving image file success !!\n'
          + '*' * 30, end='\n')
    
    # 이미지 유효성 체크
    validate_img(dir_path)

In [26]:
def do_crawling(driver_path, keywords):
    # 동시 처리할 스레드 개수 설정
    n_thread = len(keywords)
    tasks = []
    for thread_num, keyword in enumerate(keywords):
        tasks.append([driver_path, keyword, thread_num])

    with ThreadPool(n_thread) as tp:
        # 스레드에 입력할 매개변수가 2개 이상일 경우 starmap을 사용한다.
        tp.starmap(do_daum_search, tasks)
    
    
    # 종료 전 메세지
    print()
    print('=' * 60)
    
    # 폴더별 사진 수 카운트
    keyword_dirs = list(os.walk('./download'))[0][1]
    for dir in keyword_dirs:
        files= []
        path = './download/' + dir
        for _, _, files in os.walk(path):
            for file in files:
                if os.path.isfile(file):
                    files.append(file)
        print(f'{dir} : {len(files)}')
    
    print('\nAll crawling tasks success')
    print('End program')

---
## main cell
- Parameters
    - driver_path : 본인의 Chrome driver 경로
    - daum_keywords : 검색어 리스트 (6개 이하로 추천) 

In [27]:
# main

# 크롬 드라이버 경로
driver_path = 'c:/chromedriver.exe'

# 검색어 리스트 (6개 이하 추천)
daum_keywords = \
[
'남자 댄디컷'
]

do_crawling(driver_path, daum_keywords)

******************************
[thread 0: 남자 댄디컷]
******************************
[thread 0: 남자 댄디컷]
0: http://mblogthumb1.phinf.naver.net/20121128_3/tlxlzovlxkf0_1354077355287ztdEv_JPEG/%B3%B2%C0%DA_%B4%ED%B5%F0%C4%C6_%C0%E5%B5%BF%B0%C7.jpg?type=w2

[thread 0: 남자 댄디컷]
1: http://cfile240.uf.daum.net/image/160B4D054B4D54684732E0

[thread 0: 남자 댄디컷]
2: http://mblogthumb4.phinf.naver.net/20140924_118/do4club_1411484858446luvuA_JPEG/%B3%B2%C0%DA%BA%BC%B7%FD%B8%C5%C1%F7%2C%B4%ED%B5%F0%C4%C6_%281%29.JPG?type=w2

[thread 0: 남자 댄디컷]
3: http://mblogthumb3.phinf.naver.net/20130410_193/cluberhair_1365605715619JHN2n_JPEG/%B3%B2%C0%DA%B8%D3%B8%AE%BD%BA%C5%B8%C0%CF%BA%F1%B4%EB%C4%AA%B4%ED%B5%F0%C4%C6.jpg?type=w2

[thread 0: 남자 댄디컷]
4: http://mblogthumb4.phinf.naver.net/MjAxOTAzMjFfMTQx/MDAxNTUzMTQzNzIwMTU4.7J7G5kM-SBRxXR4657qi4vpMQkWcgmLtKphbUhxleBEg.-IQTLNQW8Ejpe_a2jUOhX26_HJoOX8-liDaJ4H_ZGu0g.JPEG.formenshair/%EB%8C%84%EB%94%94%EC%BB%B72.jpg?type=w966

[thread 0: 남자 댄디컷]
5: http://mblogthumb2.phinf


[thread 0: 남자 댄디컷]
48: http://cfile255.uf.daum.net/image/2046CC154B650E33438B64

[thread 0: 남자 댄디컷]
49: http://cfile210.uf.daum.net/image/1877584650077150167925

[thread 0: 남자 댄디컷]
50: http://mblogthumb1.phinf.naver.net/MjAxODA4MDhfNDIg/MDAxNTMzNzA4NTc4MTc5.glMW5mgn2jgAjBdVz63UbFMT5gPw5ijGxJ_CDWGwVTsg.y9UHJxWl7QdeTzXc-s7WqOXUqi6di9pZYnQfLlIlLkYg.JPEG.mrkingssam/%EB%82%A8%EC%9E%90%ED%97%A4%EC%96%B4%EC%8A%A4%ED%83%80%EC%9D%BC%2C%EB%8C%84%EB%94%94%EC%BB%B7%2C%EB%B3%BC%EB%A5%A8%ED%8E%8C%2C%EB%8B%A4%EC%9A%B4%ED%8E%8C_%287%29.jpg?type=w966

[thread 0: 남자 댄디컷]
51: http://mblogthumb4.phinf.naver.net/MjAxODAyMDlfMTM5/MDAxNTE4MTYwMDQxODEy.YTe0z-ul152qtsW8UpFZOS7AToo1pVdlpbC6hSc4An0g.nzauulkC2c9W0CCPXQwbFDG1nf5rR5cQodoj8xt28Kog.JPEG.rlazpsl11/skawkeoselzjt_ekswjdgksajfltmxkdlf_20180209_1111_%283%29.jpg?type=w773

[thread 0: 남자 댄디컷]
52: https://post-phinf.pstatic.net/MjAxNzA2MDJfMjQz/MDAxNDk2MzczNjIyNjA3.wiwipQJ9LTfrcqFKJKLyNXxR-HsGhTK8VcaJSHKSuBMg.undjgNrRBtjNydZU4x-VOSsZntchuu1zaOl_rcbYvxUg.JPE


[thread 0: 남자 댄디컷]
98: http://mblogthumb4.phinf.naver.net/20160506_186/annshair_1462516288479QhiED_JPEG/KakaoTalk_20160411_143704328.jpg?type=w773

[thread 0: 남자 댄디컷]
99: http://mblogthumb3.phinf.naver.net/20140924_41/do4club_1411484878056CyPYO_JPEG/%B3%B2%C0%DA%BA%BC%B7%FD%B8%C5%C1%F7%2C%B4%ED%B5%F0%C4%C6_%2811%29.JPG?type=w2

[thread 0: 남자 댄디컷]
100: http://mblogthumb3.phinf.naver.net/MjAxOTA3MDNfMjM4/MDAxNTYyMDg5NzE1ODU5.fPpQk7s1xp07gMq8r6YzRFwqSnzFwp93PlkvQeUg0g8g.j2_KdlseCrMsJsnMGWBtZYrIDc8vkvzP2QAe14Pzr8og.JPEG.kissaud1/DSC05912.JPG?type=w966

[thread 0: 남자 댄디컷]
101: http://mblogthumb4.phinf.naver.net/MjAxNzExMDdfNDMg/MDAxNTEwMDEyOTI0NTA4.8-PBdVa76IHSq9eOQEvuWmtluOIuTeHHQfwImPh9wi4g.h8pDgwfHl0ElEGPRnWK386CnOLrhbMpijXKgBOmwA74g.JPEG.cosmos3741/KakaoTalk_20171103_080044940.jpg?type=w966

[thread 0: 남자 댄디컷]
102: http://mblogthumb2.phinf.naver.net/20100615_44/mira9902_1276582014101H85Nd_jpg/2010-06-15_15;05;04_mira9902.jpg?type=w2

[thread 0: 남자 댄디컷]
103: http://mblogthumb4.phinf.nav


[thread 0: 남자 댄디컷]
140: http://mblogthumb1.phinf.naver.net/MjAxODA2MDdfODgg/MDAxNTI4MzM4ODQ3MzQ1.wyhgFxQuC7H3WBympZXq41Ix7yVl-xzaQFwz-a1ujFMg.Qn55pZovZZ84kT3CLLyE1rMWxmmff2Bjoynxgg1kxD8g.JPEG.comgea1/KakaoTalk_20180529_190401644.jpg?type=w966

[thread 0: 남자 댄디컷]
141: http://mblogthumb2.phinf.naver.net/MjAyMDA0MjhfMTIz/MDAxNTg4MDgyMTQ3Njkz.L8GIN2Pl7y_9NIQBDPkvWWGH5ZUAVpzJOkN3Zi7lfpcg.5pjhzd8AgPNlLkP4qhQQF7ChpV-CKSMqp_-r0JMfr2wg.JPEG.bluehnl/IMG_7713.JPG?type=w773

[thread 0: 남자 댄디컷]
142: http://cfile274.uf.daum.net/image/151771434F3CE40334E887

[thread 0: 남자 댄디컷]
143: http://mblogthumb4.phinf.naver.net/20130304_278/nixonlim99_1362381258165iVF19_JPEG/20130304_161303.jpg?type=w2

[thread 0: 남자 댄디컷]
144: http://mblogthumb2.phinf.naver.net/20141030_300/jooooohee2_1414658024698yADWp_PNG/%B4%ED%B5%F0%C4%C63.png?type=w2

[thread 0: 남자 댄디컷]
145: http://mblogthumb1.phinf.naver.net/MjAxOTA5MjRfMjU2/MDAxNTY5MzA0MDk3NDY5.NBnGSUyNk2DR2dBQ7TV26pk2yDEJDoRaEX7gQ1ccUqEg.-g9vynYqlwGhTzmw_-e_fINRvwJ-G2FT


[thread 0: 남자 댄디컷]
189: https://post-phinf.pstatic.net/MjAxOTAxMDNfMjc2/MDAxNTQ2NTIzMTY5NTk4.OANtwJtOWlI2vdd8JpGw2IKOGje_xDhvQIaj4E_TjL0g.mrHL9Hd2FZmhycIqpHjPVtJFW0ttMF8_yLGpyL3SMqMg.JPEG/IMG_9863.JPG?type=w1200

[thread 0: 남자 댄디컷]
190: http://cfile255.uf.daum.net/image/127A76334F44CAAD030C50

[thread 0: 남자 댄디컷]
191: http://cfile262.uf.daum.net/image/1943F6424F3CE5A50365C4

[thread 0: 남자 댄디컷]
192: http://mblogthumb2.phinf.naver.net/20150309_40/hwaryun7_1425881061022ptYaS_JPEG/%B3%B2%C0%DA_%B4%ED%B5%F0%C6%DF.JPG?type=w2

[thread 0: 남자 댄디컷]
193: http://mblogthumb3.phinf.naver.net/MjAxOTA5MjRfMTA1/MDAxNTY5MzA2MDkyMDkx.aA18D6H_1dT03jjs-by7nWSIxRvjpa4LflpW-w8zgGUg.CHnVYL_bJc9EsoLhJYwK1JPci4ymMZ32z8zyZdeMHxgg.JPEG.ri8ch8/google_co_kr_20190924_140259.jpg?type=w580

[thread 0: 남자 댄디컷]
194: http://mblogthumb1.phinf.naver.net/MjAxOTA0MDhfMTQ0/MDAxNTU0NzE4NjA1Mzgy.h3k8z-G71do9kK1O59RQ8zzrX_1A8_CzEpEGbScUNyEg.cXrTRIZ861iGhx9pcxuX4twQyCDUKKh95Txe7kzjuZEg.JPEG.houn7753/SE-65369c20-38a0-4cd5-8c71-81

...


[thread 0: 남자 댄디컷]
1402: image saved

[thread 0: 남자 댄디컷]
1403: image downloaded

[thread 0: 남자 댄디컷]
1403: image saved

[thread 0: 남자 댄디컷]
1404: image downloaded

[thread 0: 남자 댄디컷]
1404: image saved

[thread 0: 남자 댄디컷]
1405: image downloaded

[thread 0: 남자 댄디컷]
1405: image saved

[thread 0: 남자 댄디컷]
1406: image downloaded

[thread 0: 남자 댄디컷]
1406: image saved

[thread 0: 남자 댄디컷]
1407: image downloaded

[thread 0: 남자 댄디컷]
1407: image saved

[thread 0: 남자 댄디컷]
1408: image downloaded

[thread 0: 남자 댄디컷]
1408: image saved

[thread 0: 남자 댄디컷]
1409: image downloaded

[thread 0: 남자 댄디컷]
1409: image saved

[thread 0: 남자 댄디컷]
1410: image downloaded

[thread 0: 남자 댄디컷]
1410: image saved

[thread 0: 남자 댄디컷]
1411: image downloaded

[thread 0: 남자 댄디컷]
1411: image saved

[thread 0: 남자 댄디컷]
1412: image downloaded

[thread 0: 남자 댄디컷]
1412: image saved

[thread 0: 남자 댄디컷]
1413: image downloaded

[thread 0: 남자 댄디컷]
1413: image saved

[thread 0: 남자 댄디컷]
1414: image downloaded

[thread 0: 남자 댄디컷]
1414: im


[thread 0: 남자 댄디컷]
1504: image downloaded

[thread 0: 남자 댄디컷]
1504: image saved

[thread 0: 남자 댄디컷]
1505: image downloaded

[thread 0: 남자 댄디컷]
1505: image saved

[thread 0: 남자 댄디컷]
1506: image downloaded

[thread 0: 남자 댄디컷]
1506: image saved

[thread 0: 남자 댄디컷]
1507: image downloaded

[thread 0: 남자 댄디컷]
1507: image saved

[thread 0: 남자 댄디컷]
1508: image downloaded

[thread 0: 남자 댄디컷]
1508: image saved

[thread 0: 남자 댄디컷]
1509: image downloaded

[thread 0: 남자 댄디컷]
1509: image saved

[thread 0: 남자 댄디컷]
1510: image downloaded

[thread 0: 남자 댄디컷]
1510: image saved

[thread 0: 남자 댄디컷]
1511: image downloaded

[thread 0: 남자 댄디컷]
1511: image saved

[thread 0: 남자 댄디컷]
1512: image downloaded

[thread 0: 남자 댄디컷]
1512: image saved

[thread 0: 남자 댄디컷]
1513: image downloaded

[thread 0: 남자 댄디컷]
1513: image saved

[thread 0: 남자 댄디컷]
1514: image downloaded

[thread 0: 남자 댄디컷]
1514: image saved

[thread 0: 남자 댄디컷]
1515: image downloaded

[thread 0: 남자 댄디컷]
1515: image saved

[thread 0: 남자 댄디컷]
1516: im


[thread 0: 남자 댄디컷]
1605: image saved

[thread 0: 남자 댄디컷]
1606: image downloaded

[thread 0: 남자 댄디컷]
1606: image saved

[thread 0: 남자 댄디컷]
1607: image downloaded

[thread 0: 남자 댄디컷]
1607: image saved

[thread 0: 남자 댄디컷]
1608: image downloaded

[thread 0: 남자 댄디컷]
1608: image saved

[thread 0: 남자 댄디컷]
1609: image downloaded

[thread 0: 남자 댄디컷]
1609: image saved

[thread 0: 남자 댄디컷]
1610: image downloaded

[thread 0: 남자 댄디컷]
1610: image saved

[thread 0: 남자 댄디컷]
1611: image downloaded

[thread 0: 남자 댄디컷]
1611: image saved

[thread 0: 남자 댄디컷]
1612: image downloaded

[thread 0: 남자 댄디컷]
1612: image saved

[thread 0: 남자 댄디컷]
1613: image downloaded

[thread 0: 남자 댄디컷]
1613: image saved

[thread 0: 남자 댄디컷]
1614: image downloaded

[thread 0: 남자 댄디컷]
1614: image saved

[thread 0: 남자 댄디컷]
1615: image downloaded

[thread 0: 남자 댄디컷]
1615: image saved

[thread 0: 남자 댄디컷]
1616: image downloaded

[thread 0: 남자 댄디컷]
1616: image saved

[thread 0: 남자 댄디컷]
1617: image downloaded

[thread 0: 남자 댄디컷]
1617: im


[thread 0: 남자 댄디컷]
1707: image downloaded

[thread 0: 남자 댄디컷]
1707: image saved

[thread 0: 남자 댄디컷]
1708: image downloaded

[thread 0: 남자 댄디컷]
1708: image saved

[thread 0: 남자 댄디컷]
1709: image downloaded

[thread 0: 남자 댄디컷]
1709: image saved

[thread 0: 남자 댄디컷]
1710: image downloaded

[thread 0: 남자 댄디컷]
1710: image saved

[thread 0: 남자 댄디컷]
1711: image downloaded

[thread 0: 남자 댄디컷]
1711: image saved

[thread 0: 남자 댄디컷]
1712: image downloaded

[thread 0: 남자 댄디컷]
1712: image saved

[thread 0: 남자 댄디컷]
1713: image downloaded

[thread 0: 남자 댄디컷]
1713: image saved

[thread 0: 남자 댄디컷]
1714: image downloaded

[thread 0: 남자 댄디컷]
1714: image saved

[thread 0: 남자 댄디컷]
1715: image downloaded

[thread 0: 남자 댄디컷]
1715: image saved

[thread 0: 남자 댄디컷]
1716: image downloaded

[thread 0: 남자 댄디컷]
1716: image saved

[thread 0: 남자 댄디컷]
1717: image downloaded

[thread 0: 남자 댄디컷]
1717: image saved

[thread 0: 남자 댄디컷]
1718: image downloaded

[thread 0: 남자 댄디컷]
1718: image saved

[thread 0: 남자 댄디컷]
1719: im


[thread 0: 남자 댄디컷]
1808: image saved

[thread 0: 남자 댄디컷]
1809: image downloaded

[thread 0: 남자 댄디컷]
1809: image saved

[thread 0: 남자 댄디컷]
1810: image downloaded

[thread 0: 남자 댄디컷]
1810: image saved

[thread 0: 남자 댄디컷]
1811: image downloaded

[thread 0: 남자 댄디컷]
1811: image saved

[thread 0: 남자 댄디컷]
1812: image downloaded

[thread 0: 남자 댄디컷]
1812: image saved

[thread 0: 남자 댄디컷]
1813: image downloaded

[thread 0: 남자 댄디컷]
1813: image saved

[thread 0: 남자 댄디컷]
1814: image downloaded

[thread 0: 남자 댄디컷]
1814: image saved

[thread 0: 남자 댄디컷]
1815: image downloaded

[thread 0: 남자 댄디컷]
1815: image saved

[thread 0: 남자 댄디컷]
1816: image downloaded

[thread 0: 남자 댄디컷]
1816: image saved

[thread 0: 남자 댄디컷]
1817: image downloaded

[thread 0: 남자 댄디컷]
1817: image saved

[thread 0: 남자 댄디컷]
1818: image downloaded

[thread 0: 남자 댄디컷]
1818: image saved

[thread 0: 남자 댄디컷]
1819: image downloaded

[thread 0: 남자 댄디컷]
1819: image saved

[thread 0: 남자 댄디컷]
1820: image downloaded

[thread 0: 남자 댄디컷]
1820: im